<a href="https://colab.research.google.com/github/nitinsaimajji/final-year/blob/main/sms_DL_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout, LSTM, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
!pip install scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import GridSearchCV

In [37]:
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import re

# Load your training dataset
train_df = pd.read_csv('sms_train.csv')
# Load your testing dataset
test_df = pd.read_csv('sms_test.csv')


In [39]:
# Assuming test_df contains a 'label' column indicating the class (e.g., 'spam' or 'not spam')

# Separate samples by class
spam_samples_test = test_df[test_df['label'] == 'spam']
not_spam_samples_test = test_df[test_df['label'] == 'not_spam']

# Determine the number of samples to select from each class
num_samples_per_class_test = min(len(spam_samples_test), len(not_spam_samples_test))
max_total_samples = 500  # Maximum total number of samples

# Calculate the number of samples to select for test set, ensuring it doesn't exceed the maximum
num_samples_test = min(num_samples_per_class_test * 2, max_total_samples)

# Sample an equal number of instances from each class
sampled_spam_test = spam_samples_test.sample(n=num_samples_test // 2, random_state=42)
sampled_not_spam_test = not_spam_samples_test.sample(n=num_samples_test // 2, random_state=42)

# Concatenate the sampled dataframes
sampled_test_df = pd.concat([sampled_spam_test, sampled_not_spam_test])

# Shuffle the dataframe to mix spam and not spam samples
sampled_test_df = sampled_test_df.sample(frac=1, random_state=42)

print(len(sampled_test_df))


500


In [40]:
test_df=sampled_test_df

In [41]:
test_df.head()

,text,label
134,"Hi, I'm trying to create a catalog of all the ...",not_spam
687,Attention everyone!!! Check out these amazing ...,spam
2506,"Hi, I’m trying to work on a small data science...",not_spam
1852,We've got some super-duper amazing news for yo...,spam
1608,OMG THE BEST DEALS ARE HERE!!! 😍😍😍 Get ready f...,spam


In [42]:
# Assuming train_df contains a 'label' column indicating the class (e.g., 'spam' or 'not spam')

# Separate samples by class
spam_samples_train = train_df[train_df['label'] == 'spam']
not_spam_samples_train = train_df[train_df['label'] == 'not_spam']

# Determine the number of samples to select from each class
num_samples_per_class_train = min(len(spam_samples_train), len(not_spam_samples_train))
max_total_samples_train = 800  # Maximum total number of samples

# Calculate the number of samples to select for the training set, ensuring it doesn't exceed the maximum
num_samples_train = min(num_samples_per_class_train * 2, max_total_samples_train)

# Sample an equal number of instances from each class
sampled_spam_train = spam_samples_train.sample(n=num_samples_train // 2, random_state=42)
sampled_not_spam_train = not_spam_samples_train.sample(n=num_samples_train // 2, random_state=42)

# Concatenate the sampled dataframes
sampled_train_df = pd.concat([sampled_spam_train, sampled_not_spam_train])

# Shuffle the dataframe to mix spam and not spam samples
sampled_train_df = sampled_train_df.sample(frac=1, random_state=42)

print(len(sampled_train_df))


800


In [43]:
train_df=sampled_train_df
print(len(train_df))

800


In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
# Data Preprocessing
# Find the maximum number of unique words in the dataset
max_words = 100

# Find the maximum length of messages in the dataset
max_len = 50


def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Apply stemming
    ps = PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

# Preprocessing for training data
train_df['text'] = train_df['text'].apply(preprocess_text)

# Split the training dataset
X_train, y_train = train_df['text'], train_df['label']

# Tokenization
tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

# Sequence and Padding
X_train_sequences = tokenizer.texts_to_sequences(X_train)

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_len, padding='post')

# Preprocessing for testing data
test_df['text'] = test_df['text'].apply(preprocess_text)

# Split the testing dataset
X_test, y_test = test_df['text'], test_df['label']

# Tokenization
X_test_sequences = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_len, padding='post')



## Building and Training Models

## DNN

In [46]:
# dnn_model_modified = Sequential([
#     Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
#     Flatten(),
#     Dense(32, activation='sigmoid'),
#     Dropout(0.2),
#     Dense(16, activation='relu'),
#     Dropout(0.7),  # Increased dropout rate
#     Dense(1, activation='sigmoid')
# ])

def create_dnn_model(optimizer='adam',learning_rate=0.001):
    dnn_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Flatten(),
        Dense(32, activation='sigmoid'),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.7),  # Increased dropout rate
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    dnn_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return dnn_model

# Wrap the Keras model in a scikit-learn compatible classifier
dnn_classifier = KerasClassifier(build_fn=create_dnn_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_dnn = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_dnn = GridSearchCV(estimator=dnn_classifier, param_grid=param_grid_dnn, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_dnn = grid_search_dnn.fit(X_train_padded, y_train)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 2s - loss: 0.7413 - accuracy: 0.4934 - 2s/epoch - 96ms/step
Epoch 2/30
17/17 - 0s - loss: 0.7026 - accuracy: 0.5328 - 59ms/epoch - 3ms/step
Epoch 3/30
17/17 - 0s - loss: 0.6934 - accuracy: 0.5403 - 62ms/epoch - 4ms/step
Epoch 4/30
17/17 - 0s - loss: 0.7020 - accuracy: 0.5141 - 86ms/epoch - 5ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6875 - accuracy: 0.5291 - 46ms/epoch - 3ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6930 - accuracy: 0.5328 - 40ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.5178 - 33ms/epoch - 2ms/step
Epoch 8/30
17/17 - 0s - loss: 0.6804 - accuracy: 0.5422 - 48ms/epoch - 3ms/step
Epoch 9/30
17/17 - 0s - loss: 0.6755 - accuracy: 0.5835 - 44ms/epoch - 3ms/step
Epoch 10/30
17/17 - 0s - loss: 0.6466 - accuracy: 0.6135 - 47ms/epoch - 3ms/step
Epoch 11/30
17/17 - 0s - loss: 0.6227 - accuracy: 0.6492 - 55ms/epoch - 3ms/step
Epoch 12/30
17/17 - 0s - loss: 0.5617 - accuracy: 0.6998 - 79ms/epoch - 5ms/step
Epoch 13/30
17/17 - 0s - loss: 0.5224 

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


17/17 - 1s - loss: 0.7699 - accuracy: 0.5084 - 1s/epoch - 70ms/step
Epoch 2/30
17/17 - 0s - loss: 0.7405 - accuracy: 0.4991 - 43ms/epoch - 3ms/step
Epoch 3/30
17/17 - 0s - loss: 0.7126 - accuracy: 0.5197 - 52ms/epoch - 3ms/step
Epoch 4/30
17/17 - 0s - loss: 0.6995 - accuracy: 0.5441 - 41ms/epoch - 2ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6881 - accuracy: 0.5629 - 42ms/epoch - 2ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6827 - accuracy: 0.5310 - 36ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6659 - accuracy: 0.6060 - 69ms/epoch - 4ms/step
Epoch 8/30
17/17 - 0s - loss: 0.6647 - accuracy: 0.6248 - 40ms/epoch - 2ms/step
Epoch 9/30
17/17 - 0s - loss: 0.6207 - accuracy: 0.6529 - 48ms/epoch - 3ms/step
Epoch 10/30
17/17 - 0s - loss: 0.5687 - accuracy: 0.7411 - 39ms/epoch - 2ms/step
Epoch 11/30
17/17 - 0s - loss: 0.5064 - accuracy: 0.7617 - 40ms/epoch - 2ms/step
Epoch 12/30
17/17 - 0s - loss: 0.4204 - accuracy: 0.8330 - 50ms/epoch - 3ms/step
Epoch 13/30
17/17 - 0s - loss: 0.3546 - accuracy:

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

17/17 - 1s - loss: 0.8701 - accuracy: 0.4709 - 661ms/epoch - 39ms/step
Epoch 2/30
17/17 - 0s - loss: 0.7599 - accuracy: 0.5272 - 28ms/epoch - 2ms/step
Epoch 3/30
17/17 - 0s - loss: 0.7369 - accuracy: 0.4897 - 32ms/epoch - 2ms/step
Epoch 4/30
17/17 - 0s - loss: 0.7069 - accuracy: 0.5047 - 34ms/epoch - 2ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6876 - accuracy: 0.5197 - 37ms/epoch - 2ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6910 - accuracy: 0.5141 - 32ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6886 - accuracy: 0.5122 - 33ms/epoch - 2ms/step
Epoch 8/30
17/17 - 0s - loss: 0.6842 - accuracy: 0.5478 - 33ms/epoch - 2ms/step
Epoch 9/30
17/17 - 0s - loss: 0.6775 - accuracy: 0.5366 - 32ms/epoch - 2ms/step
Epoch 10/30
17/17 - 0s - loss: 0.6700 - accuracy: 0.5535 - 30ms/epoch - 2ms/step
Epoch 11/30
17/17 - 0s - loss: 0.6654 - accuracy: 0.5666 - 33ms/epoch - 2ms/step
Epoch 12/30
17/17 - 0s - loss: 0.6354 - accuracy: 0.6004 - 35ms/epoch - 2ms/step
Epoch 13/30
17/17 - 0s - loss: 0.6130 - accura

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


17/17 - 1s - loss: 0.7602 - accuracy: 0.5000 - 1s/epoch - 81ms/step
Epoch 2/30
17/17 - 0s - loss: 0.7403 - accuracy: 0.4888 - 32ms/epoch - 2ms/step
Epoch 3/30
17/17 - 0s - loss: 0.7098 - accuracy: 0.5150 - 29ms/epoch - 2ms/step
Epoch 4/30
17/17 - 0s - loss: 0.6859 - accuracy: 0.5431 - 29ms/epoch - 2ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6851 - accuracy: 0.5468 - 31ms/epoch - 2ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6637 - accuracy: 0.6255 - 31ms/epoch - 2ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6459 - accuracy: 0.5861 - 29ms/epoch - 2ms/step
Epoch 8/30
17/17 - 0s - loss: 0.5804 - accuracy: 0.6873 - 32ms/epoch - 2ms/step
Epoch 9/30
17/17 - 0s - loss: 0.5103 - accuracy: 0.7397 - 33ms/epoch - 2ms/step
Epoch 10/30
17/17 - 0s - loss: 0.4411 - accuracy: 0.7697 - 30ms/epoch - 2ms/step
Epoch 11/30
17/17 - 0s - loss: 0.3854 - accuracy: 0.8052 - 30ms/epoch - 2ms/step
Epoch 12/30
17/17 - 0s - loss: 0.3428 - accuracy: 0.8258 - 28ms/epoch - 2ms/step
Epoch 13/30
17/17 - 0s - loss: 0.3130 - accuracy:

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


25/25 - 1s - loss: 0.7334 - accuracy: 0.4963 - 665ms/epoch - 27ms/step
Epoch 2/30
25/25 - 0s - loss: 0.7037 - accuracy: 0.5238 - 39ms/epoch - 2ms/step
Epoch 3/30
25/25 - 0s - loss: 0.7063 - accuracy: 0.5263 - 40ms/epoch - 2ms/step
Epoch 4/30
25/25 - 0s - loss: 0.6920 - accuracy: 0.5288 - 38ms/epoch - 2ms/step
Epoch 5/30
25/25 - 0s - loss: 0.6745 - accuracy: 0.5675 - 48ms/epoch - 2ms/step
Epoch 6/30
25/25 - 0s - loss: 0.6538 - accuracy: 0.6237 - 56ms/epoch - 2ms/step
Epoch 7/30
25/25 - 0s - loss: 0.6295 - accuracy: 0.6388 - 60ms/epoch - 2ms/step
Epoch 8/30
25/25 - 0s - loss: 0.5077 - accuracy: 0.7788 - 53ms/epoch - 2ms/step
Epoch 9/30
25/25 - 0s - loss: 0.3922 - accuracy: 0.8450 - 56ms/epoch - 2ms/step
Epoch 10/30
25/25 - 0s - loss: 0.3087 - accuracy: 0.8963 - 51ms/epoch - 2ms/step
Epoch 11/30
25/25 - 0s - loss: 0.2196 - accuracy: 0.9250 - 51ms/epoch - 2ms/step
Epoch 12/30
25/25 - 0s - loss: 0.1891 - accuracy: 0.9438 - 50ms/epoch - 2ms/step
Epoch 13/30
25/25 - 0s - loss: 0.1908 - accura

In [47]:
# Display the results in a table
results_df_dnn = pd.DataFrame(grid_result_dnn.cv_results_)
results_df_dnn[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.985014,2
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.978767,13
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.985009,3
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.983761,5
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.982512,7
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.982512,7
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.978767,13
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.988759,1
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.983766,4
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.981264,10


In [48]:
# Print the best parameters and corresponding accuracy
print("Best Parameters (DNN): ", grid_result_dnn.best_params_)
print("Best Accuracy (DNN): ", grid_result_dnn.best_score_)

Best Parameters (DNN):  {'learning_rate': 0.001, 'optimizer': 'Adagrad'}
Best Accuracy (DNN):  0.9887593515624267


In [73]:
best_dnn_model = grid_result_dnn.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_dnn_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)

16/16 - 0s - 28ms/epoch - 2ms/step
Test Accuracy: 0.982


In [74]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='spam')
recall_spam = recall_score(y_test, y_pred, pos_label='spam')
f1_spam = f1_score(y_test, y_pred, pos_label='spam')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not_spam')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not_spam')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not_spam')

print("Precision (spam):", precision_spam)
print("Recall (spam):", recall_spam)
print("F1-score (spam):", f1_spam)

print("Precision (non-spam):", precision_non_spam)
print("Recall (non-spam):", recall_non_spam)
print("F1-score (non-spam):", f1_non_spam)


Precision (spam): 0.9958847736625515
Recall (spam): 0.968
F1-score (spam): 0.9817444219066936
Precision (non-spam): 0.9688715953307393
Recall (non-spam): 0.996
F1-score (non-spam): 0.9822485207100591


## LSTM

In [51]:
# lstm_model = Sequential([
#     Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
#     LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
#     Dropout(0.5),
#     LSTM(64),
#     Dropout(0.3),
#     Dense(32, activation='relu'),
#     Dropout(0.3),
#     Dense(16, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])


def create_lstm_model(optimizer='adam', learning_rate=0.001):
    lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
        LSTM(64, return_sequences=True),  # Added return_sequences=True for stacking LSTM layers
        Dropout(0.5),
        LSTM(64),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid_lstm = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search_lstm = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid_lstm, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result_lstm = grid_search_lstm.fit(X_train_padded, y_train)


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 4s - loss: 0.6947 - accuracy: 0.4953 - 4s/epoch - 231ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6911 - accuracy: 0.5666 - 613ms/epoch - 36ms/step
Epoch 3/30
17/17 - 1s - loss: 0.6817 - accuracy: 0.5741 - 602ms/epoch - 35ms/step
Epoch 4/30
17/17 - 1s - loss: 0.5895 - accuracy: 0.6961 - 601ms/epoch - 35ms/step
Epoch 5/30
17/17 - 1s - loss: 0.3678 - accuracy: 0.9268 - 605ms/epoch - 36ms/step
Epoch 6/30
17/17 - 1s - loss: 0.4209 - accuracy: 0.8856 - 579ms/epoch - 34ms/step
Epoch 7/30
17/17 - 1s - loss: 0.3164 - accuracy: 0.9006 - 621ms/epoch - 37ms/step
Epoch 8/30
17/17 - 1s - loss: 0.3225 - accuracy: 0.8931 - 604ms/epoch - 36ms/step
Epoch 9/30
17/17 - 1s - loss: 0.2738 - accuracy: 0.9193 - 621ms/epoch - 37ms/step
Epoch 10/30
17/17 - 1s - loss: 0.3457 - accuracy: 0.9268 - 994ms/epoch - 58ms/step
Epoch 11/30
17/17 - 1s - loss: 0.5696 - accuracy: 0.7974 - 1s/epoch - 59ms/step
Epoch 12/30
17/17 - 1s - loss: 0.5146 - accuracy: 0.7598 - 1s/epoch - 63ms/step
Epoch 13/30
17/17 - 1

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 5s - loss: 0.6921 - accuracy: 0.5347 - 5s/epoch - 305ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6882 - accuracy: 0.5704 - 931ms/epoch - 55ms/step
Epoch 3/30
17/17 - 1s - loss: 0.5825 - accuracy: 0.6904 - 837ms/epoch - 49ms/step
Epoch 4/30
17/17 - 1s - loss: 0.2874 - accuracy: 0.9418 - 814ms/epoch - 48ms/step
Epoch 5/30
17/17 - 1s - loss: 0.3544 - accuracy: 0.9156 - 574ms/epoch - 34ms/step
Epoch 6/30
17/17 - 1s - loss: 0.2176 - accuracy: 0.9437 - 555ms/epoch - 33ms/step
Epoch 7/30
17/17 - 1s - loss: 0.1087 - accuracy: 0.9794 - 560ms/epoch - 33ms/step
Epoch 8/30
17/17 - 1s - loss: 0.0916 - accuracy: 0.9775 - 579ms/epoch - 34ms/step
Epoch 9/30
17/17 - 1s - loss: 0.0872 - accuracy: 0.9869 - 571ms/epoch - 34ms/step
Epoch 10/30
17/17 - 1s - loss: 0.0771 - accuracy: 0.9812 - 571ms/epoch - 34ms/step
Epoch 11/30
17/17 - 1s - loss: 0.0553 - accuracy: 0.9887 - 575ms/epoch - 34ms/step
Epoch 12/30
17/17 - 1s - loss: 0.0505 - accuracy: 0.9906 - 569ms/epoch - 33ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
17/17 - 4s - loss: 0.6931 - accuracy: 0.5066 - 4s/epoch - 208ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6917 - accuracy: 0.4934 - 584ms/epoch - 34ms/step
Epoch 3/30
17/17 - 1s - loss: 0.5388 - accuracy: 0.7242 - 570ms/epoch - 34ms/step
Epoch 4/30
17/17 - 1s - loss: 0.2333 - accuracy: 0.9456 - 563ms/epoch - 33ms/step
Epoch 5/30
17/17 - 1s - loss: 0.2494 - accuracy: 0.9418 - 585ms/epoch - 34ms/step
Epoch 6/30
17/17 - 1s - loss: 0.1843 - accuracy: 0.9587 - 825ms/epoch - 49ms/step
Epoch 7/30
17/17 - 1s - loss: 0.2605 - accuracy: 0.9099 - 740ms/epoch - 44ms/step
Epoch 8/30
17/17 - 1s - loss: 0.1465 - accuracy: 0.9644 - 582ms/epoch - 34ms/step
Epoch 9/30
17/17 - 1s - loss: 0.1180 - accuracy: 0.9775 - 606ms/epoch - 36ms/step
Epoch 10/30
17/17 - 1s - loss: 0.0944 - accuracy: 0.9756 - 559ms/epoch - 33ms/step
Epoch 11/30
17/17 - 1s - loss: 0.1289 - accuracy: 0.9719 - 583ms/epoch - 34ms/step
Epoch 12/30
17/17 - 1s - loss: 0.0853 - accuracy: 0.9869 - 595ms/epoch - 35ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 5s - loss: 0.6928 - accuracy: 0.4963 - 5s/epoch - 266ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6882 - accuracy: 0.5674 - 685ms/epoch - 40ms/step
Epoch 3/30
17/17 - 1s - loss: 0.5461 - accuracy: 0.6891 - 772ms/epoch - 45ms/step
Epoch 4/30
17/17 - 1s - loss: 0.5484 - accuracy: 0.8202 - 968ms/epoch - 57ms/step
Epoch 5/30
17/17 - 1s - loss: 0.4967 - accuracy: 0.7416 - 717ms/epoch - 42ms/step
Epoch 6/30
17/17 - 1s - loss: 0.3656 - accuracy: 0.9195 - 684ms/epoch - 40ms/step
Epoch 7/30
17/17 - 1s - loss: 0.3324 - accuracy: 0.9213 - 694ms/epoch - 41ms/step
Epoch 8/30
17/17 - 1s - loss: 0.5517 - accuracy: 0.8277 - 707ms/epoch - 42ms/step
Epoch 9/30
17/17 - 1s - loss: 0.7611 - accuracy: 0.5674 - 689ms/epoch - 41ms/step
Epoch 10/30
17/17 - 1s - loss: 0.6683 - accuracy: 0.5412 - 676ms/epoch - 40ms/step
Epoch 11/30
17/17 - 1s - loss: 0.6628 - accuracy: 0.5506 - 706ms/epoch - 42ms/step
Epoch 12/30
17/17 - 1s - loss: 0.5703 - accuracy: 0.6704 - 696ms/epoch - 41ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
25/25 - 4s - loss: 0.6920 - accuracy: 0.5337 - 4s/epoch - 155ms/step
Epoch 2/30
25/25 - 1s - loss: 0.6571 - accuracy: 0.6150 - 841ms/epoch - 34ms/step
Epoch 3/30
25/25 - 1s - loss: 0.2804 - accuracy: 0.9438 - 831ms/epoch - 33ms/step
Epoch 4/30
25/25 - 1s - loss: 0.3952 - accuracy: 0.8988 - 838ms/epoch - 34ms/step
Epoch 5/30
25/25 - 1s - loss: 0.1849 - accuracy: 0.9650 - 823ms/epoch - 33ms/step
Epoch 6/30
25/25 - 1s - loss: 0.1564 - accuracy: 0.9663 - 837ms/epoch - 33ms/step
Epoch 7/30
25/25 - 1s - loss: 0.2773 - accuracy: 0.9287 - 830ms/epoch - 33ms/step
Epoch 8/30
25/25 - 1s - loss: 0.3136 - accuracy: 0.9013 - 817ms/epoch - 33ms/step
Epoch 9/30
25/25 - 1s - loss: 0.1992 - accuracy: 0.9500 - 842ms/epoch - 34ms/step
Epoch 10/30
25/25 - 1s - loss: 0.0927 - accuracy: 0.9800 - 840ms/epoch - 34ms/step
Epoch 11/30
25/25 - 1s - loss: 0.0582 - accuracy: 0.9837 - 807ms/epoch - 32ms/step
Epoch 12/30
25/25 - 1s - loss: 0.0373 - accuracy: 0.9912 - 920ms/epoch - 37ms/step
Epoch 13/30
25/

In [52]:
results_df_lstm = pd.DataFrame(grid_result_lstm.cv_results_)
results_df_lstm[['params', 'mean_test_score', 'rank_test_score']]

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.962458,10
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.982498,1
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.977509,4
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.976266,5
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.940061,13
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.976261,6
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.980011,3
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.948791,12
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.967470,9
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.961209,11


In [53]:
print("Best Parameters (LSTM): ", grid_result_lstm.best_params_)
print("Best Accuracy (LSTM): ", grid_result_lstm.best_score_)

Best Parameters (LSTM):  {'learning_rate': 0.01, 'optimizer': 'RMSprop'}
Best Accuracy (LSTM):  0.9824983807834192


In [75]:
best_lstm_model = grid_result_lstm.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_lstm_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)


16/16 - 0s - 168ms/epoch - 11ms/step
Test Accuracy: 0.98


In [76]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='spam')
recall_spam = recall_score(y_test, y_pred, pos_label='spam')
f1_spam = f1_score(y_test, y_pred, pos_label='spam')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not_spam')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not_spam')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not_spam')

print("Precision (spam):", precision_spam)
print("Recall (spam):", recall_spam)
print("F1-score (spam):", f1_spam)

print("Precision (non-spam):", precision_non_spam)
print("Recall (non-spam):", recall_non_spam)
print("F1-score (non-spam):", f1_non_spam)


Precision (spam): 0.9918032786885246
Recall (spam): 0.968
F1-score (spam): 0.9797570850202428
Precision (non-spam): 0.96875
Recall (non-spam): 0.992
F1-score (non-spam): 0.9802371541501976


## Bi-LSTM

In [55]:
def create_bi_lstm_model(optimizer='adam', learning_rate=0.001):
    bi_lstm_model = Sequential([
        Embedding(input_dim=max_words, output_dim=16, input_length=max_len),
        Bidirectional(LSTM(64)),
        Dense(1, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    if optimizer == 'SGD':
        opt = SGD(lr=learning_rate)
    elif optimizer == 'RMSprop':
        opt = RMSprop(lr=learning_rate)
    elif optimizer == 'Adagrad':
        opt = Adagrad(lr=learning_rate)
    elif optimizer == 'Adadelta':
        opt = Adadelta(lr=learning_rate)
    elif optimizer == 'Nadam':
        opt = Nadam(lr=learning_rate)
    else:
        opt = 'adam'

    bi_lstm_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return bi_lstm_model

# Wrap the Keras model in a scikit-learn compatible classifier
bi_lstm_classifier = KerasClassifier(build_fn=create_bi_lstm_model, epochs=30, verbose=2,learning_rate=0.01)

# Define the parameter grid for Grid Search
param_grid = {
    'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Nadam'],
    'learning_rate': [0.01, 0.001, 0.0001]
}

# Create the Grid Search object
grid_search = GridSearchCV(estimator=bi_lstm_classifier, param_grid=param_grid, scoring='accuracy', cv=3)

# Perform Grid Search on the training data
grid_result = grid_search.fit(X_train_padded, y_train)



/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 4s - loss: 0.6915 - accuracy: 0.4916 - 4s/epoch - 257ms/step
Epoch 2/30
17/17 - 0s - loss: 0.6715 - accuracy: 0.5629 - 448ms/epoch - 26ms/step
Epoch 3/30
17/17 - 0s - loss: 0.5798 - accuracy: 0.6923 - 466ms/epoch - 27ms/step
Epoch 4/30
17/17 - 0s - loss: 0.5520 - accuracy: 0.8086 - 447ms/epoch - 26ms/step
Epoch 5/30
17/17 - 0s - loss: 0.5250 - accuracy: 0.7430 - 442ms/epoch - 26ms/step
Epoch 6/30
17/17 - 0s - loss: 0.4992 - accuracy: 0.7936 - 452ms/epoch - 27ms/step
Epoch 7/30
17/17 - 0s - loss: 0.4674 - accuracy: 0.8124 - 460ms/epoch - 27ms/step
Epoch 8/30
17/17 - 0s - loss: 0.4810 - accuracy: 0.8143 - 453ms/epoch - 27ms/step
Epoch 9/30
17/17 - 0s - loss: 0.4739 - accuracy: 0.8218 - 463ms/epoch - 27ms/step
Epoch 10/30
17/17 - 0s - loss: 0.4475 - accuracy: 0.8311 - 462ms/epoch - 27ms/step
Epoch 11/30
17/17 - 0s - loss: 0.4437 - accuracy: 0.8518 - 454ms/epoch - 27ms/step
Epoch 12/30
17/17 - 0s - loss: 0.4756 - accuracy: 0.8086 - 450ms/epoch - 26ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 3s - loss: 0.6918 - accuracy: 0.5009 - 3s/epoch - 189ms/step
Epoch 2/30
17/17 - 0s - loss: 0.6715 - accuracy: 0.4991 - 402ms/epoch - 24ms/step
Epoch 3/30
17/17 - 0s - loss: 0.6041 - accuracy: 0.7242 - 421ms/epoch - 25ms/step
Epoch 4/30
17/17 - 0s - loss: 0.5161 - accuracy: 0.7711 - 408ms/epoch - 24ms/step
Epoch 5/30
17/17 - 0s - loss: 0.4860 - accuracy: 0.7917 - 401ms/epoch - 24ms/step
Epoch 6/30
17/17 - 0s - loss: 0.4897 - accuracy: 0.8274 - 404ms/epoch - 24ms/step
Epoch 7/30
17/17 - 0s - loss: 0.5591 - accuracy: 0.7861 - 383ms/epoch - 23ms/step
Epoch 8/30
17/17 - 0s - loss: 0.4614 - accuracy: 0.8105 - 416ms/epoch - 24ms/step
Epoch 9/30
17/17 - 0s - loss: 0.4438 - accuracy: 0.8462 - 399ms/epoch - 23ms/step
Epoch 10/30
17/17 - 0s - loss: 0.4533 - accuracy: 0.8236 - 398ms/epoch - 23ms/step
Epoch 11/30
17/17 - 0s - loss: 0.4388 - accuracy: 0.8424 - 422ms/epoch - 25ms/step
Epoch 12/30
17/17 - 0s - loss: 0.4361 - accuracy: 0.8386 - 395ms/epoch - 23ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/30
17/17 - 4s - loss: 0.6932 - accuracy: 0.4897 - 4s/epoch - 232ms/step
Epoch 2/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 537ms/epoch - 32ms/step
Epoch 3/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 510ms/epoch - 30ms/step
Epoch 4/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 551ms/epoch - 32ms/step
Epoch 5/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 550ms/epoch - 32ms/step
Epoch 6/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 536ms/epoch - 32ms/step
Epoch 7/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 517ms/epoch - 30ms/step
Epoch 8/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 525ms/epoch - 31ms/step
Epoch 9/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4991 - 534ms/epoch - 31ms/step
Epoch 10/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4634 - 555ms/epoch - 33ms/step
Epoch 11/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.5009 - 515ms/epoch - 30ms/step
Epoch 12/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.4728 - 537ms/epoch - 32ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
17/17 - 4s - loss: 0.6932 - accuracy: 0.4963 - 4s/epoch - 222ms/step
Epoch 2/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.5000 - 477ms/epoch - 28ms/step
Epoch 3/30
17/17 - 1s - loss: 0.6932 - accuracy: 0.5000 - 505ms/epoch - 30ms/step
Epoch 4/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.4738 - 498ms/epoch - 29ms/step
Epoch 5/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.4850 - 499ms/epoch - 29ms/step
Epoch 6/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.5000 - 477ms/epoch - 28ms/step
Epoch 7/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.4738 - 471ms/epoch - 28ms/step
Epoch 8/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.4700 - 471ms/epoch - 28ms/step
Epoch 9/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.5000 - 489ms/epoch - 29ms/step
Epoch 10/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.5000 - 490ms/epoch - 29ms/step
Epoch 11/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.5000 - 485ms/epoch - 29ms/step
Epoch 12/30
17/17 - 0s - loss: 0.6932 - accuracy: 0.5000 - 489ms/epoch - 29ms/step
Epoch 13/30
17/

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/30
25/25 - 4s - loss: 0.6932 - accuracy: 0.4875 - 4s/epoch - 176ms/step
Epoch 2/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.5000 - 613ms/epoch - 25ms/step
Epoch 3/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.5000 - 584ms/epoch - 23ms/step
Epoch 4/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.4650 - 598ms/epoch - 24ms/step
Epoch 5/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.5025 - 615ms/epoch - 25ms/step
Epoch 6/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.5000 - 596ms/epoch - 24ms/step
Epoch 7/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.5000 - 597ms/epoch - 24ms/step
Epoch 8/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.5000 - 592ms/epoch - 24ms/step
Epoch 9/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.5000 - 604ms/epoch - 24ms/step
Epoch 10/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.4800 - 574ms/epoch - 23ms/step
Epoch 11/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.4800 - 613ms/epoch - 25ms/step
Epoch 12/30
25/25 - 1s - loss: 0.6932 - accuracy: 0.4650 - 606ms/epoch - 24ms/step
Epoch 13/30
25/

In [56]:
results_df = pd.DataFrame(grid_result.cv_results_)
results_df[['params', 'mean_test_score', 'rank_test_score']]
# Print the best parameters and corresponding accuracy

,params,mean_test_score,rank_test_score
0,"{'learning_rate': 0.01, 'optimizer': 'SGD'}",0.778402,4
1,"{'learning_rate': 0.01, 'optimizer': 'RMSprop'}",0.658552,9
2,"{'learning_rate': 0.01, 'optimizer': 'Adagrad'}",0.970023,1
3,"{'learning_rate': 0.01, 'optimizer': 'Adadelta'}",0.965011,2
4,"{'learning_rate': 0.01, 'optimizer': 'Nadam'}",0.654806,11
5,"{'learning_rate': 0.001, 'optimizer': 'SGD'}",0.817104,3
6,"{'learning_rate': 0.001, 'optimizer': 'RMSprop'}",0.498752,12
7,"{'learning_rate': 0.001, 'optimizer': 'Adagrad'}",0.659800,7
8,"{'learning_rate': 0.001, 'optimizer': 'Adadelta'}",0.662912,5
9,"{'learning_rate': 0.001, 'optimizer': 'Nadam'}",0.662297,6


In [57]:
print("\n",grid_result)
print("\n")
print("Best Parameters: ", grid_result.best_params_)
print("Best Accuracy: ", grid_result.best_score_)



 GridSearchCV(cv=3,
             estimator=KerasClassifier(build_fn=<function create_bi_lstm_model at 0x7e1a0cf3ee60>, epochs=30, learning_rate=0.01, verbose=2),
             param_grid={'learning_rate': [0.01, 0.001, 0.0001],
                         'optimizer': ['SGD', 'RMSprop', 'Adagrad', 'Adadelta',
                                       'Nadam']},
             scoring='accuracy')


Best Parameters:  {'learning_rate': 0.01, 'optimizer': 'Adagrad'}
Best Accuracy:  0.9700233730393398


In [71]:
best_model = grid_result.best_estimator_
from sklearn.metrics import accuracy_score

# Predict classes for the testing data
y_pred = best_model.predict(X_test_padded)

# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred)

print("Test Accuracy:", test_accuracy)

16/16 - 0s - 123ms/epoch - 8ms/step
Test Accuracy: 0.5


In [72]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming y_true contains true labels and y_pred contains predicted labels
# Calculate precision, recall, and F1-score for both classes
precision_spam = precision_score(y_test, y_pred, pos_label='spam')
recall_spam = recall_score(y_test, y_pred, pos_label='spam')
f1_spam = f1_score(y_test, y_pred, pos_label='spam')

precision_non_spam = precision_score(y_test, y_pred, pos_label='not_spam')
recall_non_spam = recall_score(y_test, y_pred, pos_label='not_spam')
f1_non_spam = f1_score(y_test, y_pred, pos_label='not_spam')

print("Precision (spam):", precision_spam)
print("Recall (spam):", recall_spam)
print("F1-score (spam):", f1_spam)

print("Precision (non-spam):", precision_non_spam)
print("Recall (non-spam):", recall_non_spam)
print("F1-score (non-spam):", f1_non_spam)


Precision (spam): 0.5
Recall (spam): 1.0
F1-score (spam): 0.6666666666666666
Precision (non-spam): 0.0
Recall (non-spam): 0.0
F1-score (non-spam): 0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
